# huMon Generator

In [1]:
# !pip install jupyter-dash
from jupyter_dash import JupyterDash  # Use JupyterDash instead of Dash for Jupyter environments
import networkx as nx
import pandas as pd
import random
import numpy as np
import plotly.graph_objs as go
from matplotlib.colors import to_hex, to_rgb
from dash import dcc, html, Input, Output

class StatTreeGenerator:
    def __init__(self, branch_pattern):
        self.branch_pattern = branch_pattern
        self.nodes = {}
        self.G = nx.DiGraph()
        
        # Color codes for Generation 1 nodes
        self.colors = {
            "0-0": 'magenta',
            "0-1": 'green',
            "0-2": 'yellow',
            "0-3": 'cyan',
            "0-4": 'black',
            "0-5": 'purple'
        }
        self.max_stats_per_generation = {}

    def generate_tree(self):
        node_id = "0"
        # Generation 0: initialize root node with base stats totaling 100
        base_stat = 100 // 6
        self.nodes[node_id] = {
            "generation": 0,
            "stats": {stat: base_stat for stat in ["Health", "PAttack", "PDefense", "MAttack", "MDefense", "Speed"]},
            "color": 'white'
        }
        self.G.add_node(node_id)
        self.create_branches(node_id, 1)
        
        # Calculate max stats for each generation
        self.calculate_max_stats_per_generation()

        return pd.DataFrame([{"node_id": node, **self.nodes[node]} for node in self.nodes])

    def calculate_max_stats_per_generation(self):
        """Calculate the maximum possible stat for each generation based on the doubling point system."""
        fibonacci_weights = [13, 8, 5, 3, 2, 1]
        total_weight = sum(fibonacci_weights)
        
        for generation in range(1, max(self.branch_pattern) + 1):
            min_points = 64 * (2 ** (generation - 1))
            max_points = 300 * (2 ** (generation - 1))
            max_additional_points = max_points
            
            # Calculate the maximum possible distribution for this generation
            stat_increase = [int(max_additional_points * weight / total_weight) for weight in fibonacci_weights]
            max_stat_increase = sum(stat_increase)
            self.max_stats_per_generation[generation] = max_stat_increase + 100  # Starting base stats + max increment

    def create_branches(self, parent_id, generation):
        if generation > len(self.branch_pattern):
            return
        num_children = self.branch_pattern[generation - 1]
        for i in range(num_children):
            child_id = f"{parent_id}-{i}"
            
            if generation == 1:
                # Generation 1 nodes with Fibonacci-based stat distribution
                color = self.colors.get(child_id, 'gray')
                stats = self.generate_ranked_stats(generation)
            else:
                # Subsequent generations with adjusted stats
                color = self.adjust_color(self.nodes[parent_id]["color"], generation)
                stats = self.specialize_ranked_stats(parent_id, generation)

            self.nodes[child_id] = {"generation": generation, "stats": stats, "color": color}
            self.G.add_edge(parent_id, child_id)
            self.create_branches(child_id, generation + 1)

    def generate_ranked_stats(self, generation):
            # Determine additional points range based on generation using a doubling formula
            min_points = 64 * (2 ** (generation - 1))
            max_points = 128 * (2 ** (generation - 1))
            additional_points = random.randint(min_points, max_points)
            
            # Define Fibonacci weights for distribution
            fibonacci_weights = [13, 8, 5, 3, 2, 1]
            total_weight = sum(fibonacci_weights)
            base_increase = [int(additional_points * weight / total_weight) for weight in fibonacci_weights]

            # Shuffle stats order and prepare a flexible assignment
            stat_order = ["Health", "PAttack", "PDefense", "MAttack", "MDefense", "Speed"]
            random.shuffle(stat_order)
            
            # Distribute points with some added randomization for variability
            stats = {}
            for i, stat in enumerate(stat_order):
                # Introduce potential multiplier for variability, especially for top stats
                multiplier = random.uniform(1.0, 1.5) if i < 2 else random.uniform(0.8, 1.2)
                stats[stat] = self.nodes["0"]["stats"][stat] + int(base_increase[i] * multiplier)
            
            return stats

    def specialize_ranked_stats(self, parent_id, generation):
        # Determine additional points range based on generation using a doubling formula
        min_points = 64 * (2 ** (generation - 1))
        max_points = 128 * (2 ** (generation - 1))
        additional_points = random.randint(min_points, max_points)

        parent_stats = self.nodes[parent_id]["stats"]
        
        # 10-25% of additional points go to a few of the top parent stats instead of just one
        high_stat_candidates = sorted(parent_stats, key=parent_stats.get, reverse=True)[:3]
        boost_points = int(additional_points * random.uniform(0.1, 0.25))
        
        stats = {stat: parent_stats[stat] for stat in parent_stats}
        
        # Distribute boost points among multiple high stats to allow for more extreme stats
        for stat in high_stat_candidates:
            stats[stat] += int(boost_points / len(high_stat_candidates))

        # Remaining points distributed among stats with more random variation
        remaining_points = additional_points - boost_points
        stat_order = list(parent_stats.keys())
        random.shuffle(stat_order)
        
        for stat in stat_order:
            points = int(remaining_points / len(stat_order) * random.uniform(0.8, 1.2))
            stats[stat] += points
            remaining_points -= points
        
        return stats

    def adjust_color(self, color, generation):
        rgb = np.array(to_rgb(color))
        factor = 0.8 + (generation * 0.05)
        return to_hex(np.clip(rgb * factor, 0, 1))

    def hierarchy_pos(self, G, root="0", width=1.0, vert_gap=0.4, vert_loc=0.5, xcenter=0.5):
        """Generate hierarchical positions for each node in the tree."""
        pos = {}

        def _hierarchy_pos(node, left, right, vert_loc, parent=None):
            pos[node] = ((left + right) / 2, vert_loc)
            children = list(G.successors(node))
            if children:
                dx = (right - left) / len(children)
                nextx = left
                for child in children:
                    _hierarchy_pos(child, nextx, nextx + dx, vert_loc - vert_gap, node)
                    nextx += dx

        _hierarchy_pos(root, 0, width, vert_loc)
        return pos

    def plot_tree_structure(self):
        pos = self.hierarchy_pos(self.G)
        edge_x, edge_y = [], []
        for edge in self.G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')
        
        node_x, node_y, hover_text, marker_colors, node_ids = [], [], [], [], []
        for node in self.G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            stats = self.nodes[node]["stats"]
            color = self.nodes[node]["color"]
            text = f"Node ID: {node}<br>Generation: {self.nodes[node]['generation']}"
            hover_text.append(text)
            marker_colors.append(color)
            node_ids.append(node)  # Add node ID to node_ids list

        # Assign node IDs to customdata
        node_trace = go.Scatter(
            x=node_x, y=node_y, mode='markers', hoverinfo='text', text=hover_text,
            marker=dict(size=20, color=marker_colors, line=dict(color='black', width=1)),
            customdata=node_ids, name="tree"  # Attach node IDs to customdata
        )
        
        return [edge_trace, node_trace]

# Initialize the tree
tree_generator = StatTreeGenerator(branch_pattern=[6, 3, 3])
tree_generator.generate_tree()

# Initialize JupyterDash app
app = JupyterDash(__name__)  # Change to JupyterDash

app.layout = html.Div([
    # Ancestral tree on the left (75% width)
    html.Div([
        dcc.Graph(id='tree-plot', figure={'data': tree_generator.plot_tree_structure(), 'layout': go.Layout(
            title='Ancestral Tree', showlegend=False, margin=dict(b=20, l=20, r=20, t=40))}),
    ], style={'width': '75%', 'display': 'inline-block', 'vertical-align': 'top'}),
    
    # Radar chart and cumulative stats on the right (25% width)
    html.Div([
        dcc.Graph(id='radar-chart'),
        html.Div(id='cumulative-stats', style={'margin-top': '20px', 'font-size': '16px'}),
    ], style={'width': '25%', 'display': 'inline-block', 'vertical-align': 'top'})
])

@app.callback(
    [Output('radar-chart', 'figure'), Output('cumulative-stats', 'children')],
    Input('tree-plot', 'clickData')
)
def update_radar_chart(clickData):
    if not clickData:
        return go.Figure(), "Select a node to view stats"
    
    node_id = clickData['points'][0].get('customdata')
    if not node_id:
        return go.Figure(), "Node data not found"

    stats = tree_generator.nodes[node_id]["stats"]

    # Define a fixed order for the stats
    fixed_order = ["Health", "PAttack", "PDefense", "MAttack", "MDefense", "Speed"]
    values = [stats[stat] for stat in fixed_order]
    
    # Find the max stat for scaling the radar chart
    max_stat_for_node = max(values)
    
    radar_chart = go.Figure()
    radar_chart.add_trace(go.Scatterpolar(
        r=values,
        theta=fixed_order,  # Use the fixed order for theta
        fill='toself',
        marker=dict(color=tree_generator.nodes[node_id]["color"])
    ))
    radar_chart.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, max_stat_for_node])),
        showlegend=False,
        title=f"Stats for Node {node_id}"
    )
    
    # Cumulative stats display
    cumulative_stats = f"Cumulative Stats for {node_id}: " + ", ".join([f"{stat}: {value}" for stat, value in zip(fixed_order, values)])
    return radar_chart, cumulative_stats


# Run the app in Jupyter notebook mode
# Run the app in Jupyter notebook mode only if it's not already running
try:
    app.run_server(mode="inline", debug=True)
except RuntimeError as e:
    if "address already in use" in str(e):
        print("The server is already running.")

/opt/anaconda3/lib/python3.12/site-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

